In [2]:
install sklearn

ModuleNotFoundError: No module named 'sklearn'

In [1]:
%run data_preprocessing.ipynb

ModuleNotFoundError: No module named 'sklearn'

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import os
import os.path
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
# for get_data
enable_feature_engineering = True
enable_outlier_handling = True
normalize = 'minmax'
use_one_hot_encoding = True
split_size = (0.9, 0.0, 0.1)

In [ ]:
y_train, x_train, y_val, x_val, y_test, x_test = get_data(enable_feature_engineering, enable_outlier_handling, normalize, use_one_hot_encoding, split_size)

In [ ]:
#compute the confusion matrix
cnf_matrix = confusion_matrix(golf_target_test, golf_test_prediction)
print(cnf_matrix)

#compute accuracy score
print("Accuracy: {}".format(accuracy_score(golf_target_test, golf_test_prediction)))

#plot the confusion matrix
plot_confusion_matrix(cnf_matrix, classes=label_encoder.classes_, title='Naive Bayes Classifier')